In [0]:
# This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. DBFS is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

# This notebook is written in Python so the default cell type is Python. However, you can use different languages by using the %LANGUAGE syntax. Python, Scala, SQL, and R are all supported

In [0]:
%run /movie-recommendation/authorization

In [0]:
import datetime
import pyspark.sql.functions as f
import pyspark.sql.types 
import pandas as pd 

from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import Window
from pyspark.sql.functions import rank, min 

Out[2]: [FileInfo(path='abfss://source-data@sgprojectpro.dfs.core.windows.net/links.csv', name='links.csv', size=1368578),
 FileInfo(path='abfss://source-data@sgprojectpro.dfs.core.windows.net/movies.csv', name='movies.csv', size=3038099),
 FileInfo(path='abfss://source-data@sgprojectpro.dfs.core.windows.net/ratings.csv', name='ratings.csv', size=678260987),
 FileInfo(path='abfss://source-data@sgprojectpro.dfs.core.windows.net/tags.csv', name='tags.csv', size=38810332)]

In [0]:
# File location and type
file_location = "abfss://source-data@sgprojectpro.dfs.core.windows.net/movies.csv"

file_type = "csv"



# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
 
df_movies = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(file_location)
    
display(df_movies)

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


movieId,imdbId,tmdbId
1,114709,862
2,113497,8844
3,113228,15602
4,114885,31357
5,113041,11862
6,113277,949
7,114319,11860
8,112302,45325
9,114576,9091
10,113189,710


In [0]:
# Create a view or table

temp_table_name = "movies_csv"

df_movies.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `movies_csv`

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "listmovies2_csv"

df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
#Read data "link"
links="abfss://source-data@sgprojectpro.dfs.core.windows.net/links.csv"
df_links = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(links)
    
display(df_links)

movieId,imdbId,tmdbId
1,114709,862
2,113497,8844
3,113228,15602
4,114885,31357
5,113041,11862
6,113277,949
7,114319,11860
8,112302,45325
9,114576,9091
10,113189,710


In [0]:
#Read data "tags"
tags="abfss://source-data@sgprojectpro.dfs.core.windows.net/tags.csv"
df_tags = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(tags)
    
display(df_tags)

userId,movieId,tag,timestamp
3,260,classic,1439472355
3,260,sci-fi,1439472256
4,1732,dark comedy,1573943598
4,1732,great dialogue,1573943604
4,7569,so bad it's good,1573943455
4,44665,unreliable narrators,1573943619
4,115569,tense,1573943077
4,115713,artificial intelligence,1573942979
4,115713,philosophical,1573943033
4,115713,tense,1573943042


In [0]:
#Read data "ratings"
ratings="abfss://source-data@sgprojectpro.dfs.core.windows.net/ratings.csv"
df_ratings = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(ratings)
    
display(df_ratings)


userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839
1,1250,4.0,1147868414


In [0]:
#count of records
df_movies.count()


Out[13]: 62423

In [0]:
#Filter out movies having rating > 3, first join movies and ratings tables
df_movies_with_ratings = df_movies.join(df_ratings,'movieId','left')

display(df_movies_with_ratings)

movieId,title,genres,userId,rating,timestamp
26,Othello (1995),Drama,34,3.0,1317760863
26,Othello (1995),Drama,90,3.0,863538343
26,Othello (1995),Drama,100,4.0,862168751
26,Othello (1995),Drama,179,4.0,849157930
26,Othello (1995),Drama,322,2.0,987392101
26,Othello (1995),Drama,325,4.0,868716756
26,Othello (1995),Drama,407,5.0,859254239
26,Othello (1995),Drama,576,5.0,843057851
26,Othello (1995),Drama,599,4.0,949272715
26,Othello (1995),Drama,621,3.0,875723591


In [0]:
#Check if there are no duplicates
df_movies_no_dups = df_movies_with_ratings.groupby('movieId').count()
display(df_movies_no_dups)

movieId,count
26,2549
27,1577
28,2950
31,9106
34,31456
44,10629
53,137
65,4663
76,3235
78,1129


In [0]:
#Join with users dataset
df_movies_with_ratings=df_movies_with_ratings.join(df_tags,['movieID','userId'],'inner')
display(df_movies_with_ratings)

movieId,userId,title,genres,rating,timestamp,tag,timestamp
1,40187,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5,1271465920,buddy movie,1271465930
1,40187,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5,1271465920,Tom Hanks,1271465933
1,40187,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5,1271465920,witty,1271465935
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,cute,1337209280
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,funny,1337209280
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,story,1337209279
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,voice acting,1337209280
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,witty,1337209280
1,51799,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1445206628,adventure,1445458888
1,51799,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1445206628,animation,1445458806


In [0]:
#Join with ratings with tag
df_ratings_tags=df_ratings.join(df_tags,['movieID'],'inner')
display(df_movies_with_ratings)

movieId,userId,title,genres,rating,timestamp,tag,timestamp
1,40187,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5,1271465920,buddy movie,1271465930
1,40187,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5,1271465920,Tom Hanks,1271465933
1,40187,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.5,1271465920,witty,1271465935
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,cute,1337209280
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,funny,1337209280
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,story,1337209279
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,voice acting,1337209280
1,48627,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1337188285,witty,1337209280
1,51799,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1445206628,adventure,1445458888
1,51799,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1445206628,animation,1445458806


In [0]:
 #convert int to string first
df_ratings=df_ratings.withColumn("tsDate", f.from_unixtime("timestamp")) 
display(df_ratings) 

userId,movieId,rating,timestamp,tsDate
1,296,5.0,1147880044,2006-05-17 15:34:04
1,306,3.5,1147868817,2006-05-17 12:26:57
1,307,5.0,1147868828,2006-05-17 12:27:08
1,665,5.0,1147878820,2006-05-17 15:13:40
1,899,3.5,1147868510,2006-05-17 12:21:50
1,1088,4.0,1147868495,2006-05-17 12:21:35
1,1175,3.5,1147868826,2006-05-17 12:27:06
1,1217,3.5,1147878326,2006-05-17 15:05:26
1,1237,5.0,1147868839,2006-05-17 12:27:19
1,1250,4.0,1147868414,2006-05-17 12:20:14


In [0]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[19]: DataFrame[key: string, value: string]

In [0]:
#String to Date Conversion 
df_ratings=df_ratings.select('userId','movieId','rating',f.to_date(unix_timestamp('tsDate','yyyy-MM-dd HH:mm:ss').cast('timestamp')).alias('rating_date'))

In [0]:
display(df_ratings)

userId,movieId,rating,rating_date
1,296,5.0,2006-05-17
1,306,3.5,2006-05-17
1,307,5.0,2006-05-17
1,665,5.0,2006-05-17
1,899,3.5,2006-05-17
1,1088,4.0,2006-05-17
1,1175,3.5,2006-05-17
1,1217,3.5,2006-05-17
1,1237,5.0,2006-05-17
1,1250,4.0,2006-05-17


In [0]:

df_ratings_year=df_ratings.groupBy('rating_date').count()
display(df_ratings_year) 

rating_date,count
2006-05-17,2946
2015-05-19,4747
2002-03-02,1861
2013-01-22,1819
2013-09-09,2147
2018-08-10,3588
2014-11-12,1964
2017-09-11,3521
2017-08-11,3355
2014-09-26,929


In [0]:
df_avg_ratings=df_ratings.groupBy('movieId').mean('rating') 
display(df_avg_ratings) 




movieId,avg(rating)
1088,3.25002094679514
1580,3.5817083457378187
3175,3.6077836141619484
44022,3.2593627146699773
175197,2.754918032786885
1645,3.547347362181387
471,3.6579813752234034
3794,3.247051114023591
8638,3.9717508278145695
33722,3.5552486187845305


In [0]:
df=df_avg_ratings.join(df_movies,'movieId','inner')
df=df.withColumnRenamed('avg(rating)','avg_rating') 


In [0]:
display(df)

movieId,avg_rating,title,genres
1088,3.25002094679514,Dirty Dancing (1987),Drama|Musical|Romance
1580,3.5817083457378187,Men in Black (a.k.a. MIB) (1997),Action|Comedy|Sci-Fi
3175,3.6077836141619484,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi
44022,3.2593627146699773,Ice Age 2: The Meltdown (2006),Adventure|Animation|Children|Comedy
175197,2.754918032786885,The Dark Tower (2017),Fantasy|Horror|Sci-Fi|Western
1645,3.547347362181387,The Devil's Advocate (1997),Drama|Mystery|Thriller
471,3.6579813752234034,"Hudsucker Proxy, The (1994)",Comedy
3794,3.247051114023591,Chuck & Buck (2000),Comedy|Drama
8638,3.9717508278145695,Before Sunset (2004),Drama|Romance
33722,3.5552486187845305,Ladies in Lavender (2004),Comedy|Drama|Romance


In [0]:
df_total_rating=df_ratings.groupBy('movieId').count() 
display(df_total_rating) 

movieId,count
1088,11935
1580,40308
3175,14659
44022,4833
175197,610
1645,13496
471,10631
3794,763
8638,4832
33722,181


In [0]:

#Filter out movies which has number of ratings less than 5  
df_total_rating=df_total_rating.filter(df_total_rating['count']>5) 
df_ratings_filtered=df_ratings.join(df_total_rating,'movieId','inner') 

In [0]:
df_total_rating.count()

Out[28]: 30207

In [0]:
df_rating_per_user = df_ratings_filtered.select('userId','movieId','rating').groupBy('userId','movieId').max('rating')

df_rating_per_user_movie=df_rating_per_user.join(df_movies,'movieId','inner')

In [0]:
df_rating_per_user_movie=df_rating_per_user_movie.withColumnRenamed('max(rating)','max_rating')


display(df_rating_per_user_movie) 

movieId,userId,max_rating,title,genres
56022,23453,4.0,Harrison Bergeron (1995),Drama|Sci-Fi
82459,23453,3.0,True Grit (2010),Western
176751,23453,4.0,American Made (2017),Crime|Thriller
1270,23458,5.0,Back to the Future (1985),Adventure|Comedy|Sci-Fi
3000,23458,4.5,Princess Mononoke (Mononoke-hime) (1997),Action|Adventure|Animation|Drama|Fantasy
1270,23460,5.0,Back to the Future (1985),Adventure|Comedy|Sci-Fi
516,23461,3.5,Renaissance Man (1994),Comedy|Drama
1270,23461,4.5,Back to the Future (1985),Adventure|Comedy|Sci-Fi
3000,23461,4.5,Princess Mononoke (Mononoke-hime) (1997),Action|Adventure|Animation|Drama|Fantasy
82459,23461,3.0,True Grit (2010),Western


In [0]:
df_rating=df_rating_per_user_movie.groupBy('userId','movieId',"title","genres").max('max_rating')

In [0]:
display(df_rating)

userId,movieId,title,genres,max(max_rating)
31572,597,Pretty Woman (1990),Comedy|Romance,3.5
31574,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,5.0
31574,2797,Big (1988),Comedy|Drama|Fantasy|Romance,4.0
31576,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.5
31576,1653,Gattaca (1997),Drama|Sci-Fi|Thriller,3.5
31576,6157,Daredevil (2003),Action|Crime,3.0
31576,6188,Old School (2003),Comedy,2.5
31578,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,3.5
31578,597,Pretty Woman (1990),Comedy|Romance,2.0
31578,1199,Brazil (1985),Fantasy|Sci-Fi,1.0


In [0]:
#users with movies with > 4 ratings 
df_rating=df_rating.withColumnRenamed('max(max_rating)','max_rating') 

df_rating=df_rating.filter(df_rating['max_rating']>=4)

display(df_rating) 


userId,movieId,title,genres,max_rating
31574,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,5.0
31574,2797,Big (1988),Comedy|Drama|Fantasy|Romance,4.0
31576,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.5
31581,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,5.0
31581,597,Pretty Woman (1990),Comedy|Romance,5.0
31582,1653,Gattaca (1997),Drama|Sci-Fi|Thriller,4.0
31584,1199,Brazil (1985),Fantasy|Sci-Fi,4.5
31584,1212,"Third Man, The (1949)",Film-Noir|Mystery|Thriller,5.0
31584,148626,"Big Short, The (2015)",Drama,4.0
31585,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.5


In [0]:
#Identify best movies per genre
df_movies_per_genre=df_rating.groupBy('genres','title').count()

display(df_movies_per_genre)

genres,title,count
Drama,Margot at the Wedding (2007),78
Drama|Romance,"Reader, The (2008)",1437
Thriller|Western,Pursued (1947),14
Drama|Mystery|Thriller,Insomnia (1997),773
Comedy|Drama,American Splendor (2003),2299
Drama|Romance,Two Days (2011),6
Horror,Terror Train (1980),14
Comedy,Ladybugs (1992),11
Comedy,Hardbodies (1984),3
Comedy,It's Love I'm After (1937),4


In [0]:
#identify genre of user
df_rating_genre=df_rating.select('userId','title','genres').groupBy('userId','genres').count()

In [0]:
display(df_rating_genre)

userId,genres,count
31798,Crime|Horror|Thriller,1
84,Comedy|Drama,5
1203,Action|Fantasy|Horror,2
1390,Comedy,13
2207,Animation|Comedy,1
4974,Drama,36
5242,Drama|Romance,3
8296,Animation|Comedy,2
8472,Action|Fantasy|Horror,1
9996,Adventure|Animation|Children|Comedy|Fantasy,2


In [0]:
df_ratings.head(10)

Out[41]: [Row(userId=1, movieId=296, rating=5.0, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=306, rating=3.5, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=307, rating=5.0, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=665, rating=5.0, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=899, rating=3.5, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=1088, rating=4.0, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=1175, rating=3.5, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=1217, rating=3.5, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=1237, rating=5.0, rating_date=datetime.date(2006, 5, 17)),
 Row(userId=1, movieId=1250, rating=4.0, rating_date=datetime.date(2006, 5, 17))]

In [0]:
df_recent_movie=df_ratings.groupBy('userId','movieId').agg(f.max(df_ratings['rating_date']))

In [0]:
display(df_recent_movie)

userId,movieId,max(rating_date)
3,5004,2017-01-18
3,6754,2015-08-13
3,35836,2016-01-27
3,87232,2015-08-13
4,3624,2019-11-16
4,122882,2019-11-16
5,1246,1997-03-17
6,2028,1999-12-14
8,1777,1998-03-21
11,48043,2008-04-12


In [0]:
#latest trensdings overall
df_ratings_per_genre=df.groupBy('genres').avg('avg_rating')
display(df_ratings_per_genre)

genres,avg(avg_rating)
Action|Drama|Horror,2.5160323719980475
Action|Adventure|Drama|Fantasy,2.8001196021897865
Comedy|Horror|Thriller,2.77756923518892
Comedy|Drama|Horror|Thriller,2.755952380952381
Action|Animation|Comedy|Sci-Fi,3.301587301587302
Adventure|Sci-Fi|Thriller,2.4521332424668367
Children|Comedy|Drama|Fantasy|Sci-Fi,3.1875
Action|Adventure|Comedy|Horror|Thriller,3.125
Adventure|Sci-Fi,2.663536164459448
Adventure|Animation,2.8561197834498473
